Prediction clones of fedulya user from best models created in fedulya_classification.ipynb, averaging by all models, finding clones and quick review of user-clones.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jan  5 00:40:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
import os
import time

import pandas as pd
import numpy as np

In [ ]:
!pip install ktrain

In [ ]:
import ktrain
from ktrain import text

In [20]:
Data = '/content/drive/MyDrive/Colab Notebooks/Projects/eva/Data/'

test_Messages_filename='all_fedulya_and_clones_messages.csv'
test_Messages_full_filename= os.path.join(Data, test_Messages_filename)

Authors_filename='Authors.csv'
Authors_full_filename= os.path.join(Data, Authors_filename)

CoreAuthors_filename='CoreAuthors.csv'
CoreAuthors_full_filename=os.path.join(Data, CoreAuthors_filename)

text_column='original_message'
classname='fedulya'
prediction_column='prediction'

Models = '/content/drive/MyDrive/Colab Notebooks/Projects/eva/Models/fedulya/'

threshold_fedulya_probability=0.9
threshold_frc_fedulya_messages=0.7

## Data for prediction

In [5]:
df=pd.read_csv(test_Messages_full_filename, error_bad_lines=False, index_col=False)

In [ ]:
len(df)

435070

In [ ]:
df.head()

,Message_Id,Timestamp,Topic_1st_Message,Parent_Id,Author_Id,Topic_Id,Topic,Chapter_Id,Chapter,original_message,message,cnt_childs,cnt_immediate_childs,author,prediction_0,prediction_1,prediction_2,prediction_3,prediction_4,prediction_5,prediction_6,prediction_7,prediction_8,prediction_9,prediction
0,68156918,2011-07-20 15:29:00,N,0,240561,1588239,Ссылки,137,Детская психология и развитие,"<div class=""body"">Я все могу, где скачать? И б...","Я все могу, где скачать? И беби энштейн? Что-т...",0,0,Олечкая **,0.323189,0.086883,0.430139,0.152121,0.171385,0.075510,0.021670,0.428686,0.088370,0.317306,0.209526
1,65203211,2011-03-19 15:06:00,N,65031236,240561,2604407,методичка по развитию,137,Детская психология и развитие,"<div class=""body"">а можно мне тоже опыты vidi2...",а можно мне тоже опыты vidi2005olga@mail.ru за...,0,0,Олечкая **,0.000114,0.000092,0.000114,0.000091,0.000117,0.000118,0.000095,0.000095,0.000097,0.000098,0.000103
2,72508255,2012-01-22 16:37:00,N,0,240561,2859257,заикание,137,Детская психология и развитие,"<div class=""body"">мы ездили в Арлилию, это в М...","мы ездили в Арлилию, это в Медведково. На конс...",0,0,Олечкая **,0.000125,0.000096,0.000090,0.000096,0.000106,0.000094,0.000128,0.000094,0.000102,0.000099,0.000103
3,81885636,2013-08-29 09:39:00,N,81885143,240561,3159289,Мы напрягаем родителей?,137,Детская психология и развитие,"<div class=""body"">Как мама дочки, скажу. Мне б...","Как мама дочки, скажу. Мне было бы приятно.)))...",2,1,Олечкая **,0.000113,0.000085,0.000078,0.000093,0.000098,0.000080,0.000097,0.000082,0.000095,0.000081,0.000090
4,81888444,2013-08-29 12:57:00,N,81888288,240561,3159289,Мы напрягаем родителей?,137,Детская психология и развитие,"<div class=""body"">+1<br/><br/>Не в обиду, но к...","+1Не в обиду, но когда тут пишут, что я фен не...",10,1,Олечкая **,0.000139,0.000140,0.000118,0.000106,0.000128,0.000089,0.000116,0.000100,0.000120,0.000093,0.000115


## Prediction clone or not

In [ ]:
prediction_column_lists=list()
ts_start = time.time()
for m in next(os.walk(Models))[1]:  
  ind= m.split('_')[len(m.split('_'))-1]
  prediction_column_lists.append('%s_%s'%(prediction_column,ind))
  #the notebook may be interruoted due to the lack of memory and can be restarted several times to predict from all models
  if '%s_%s'%(prediction_column,ind) in df.columns:
    print('%s model is already processed'%m)
    continue
  print('Processing %s model'%m)
  Model_foldername = m
  Model_full_foldername = os.path.join(Models, Model_foldername)

  predictor = ktrain.load_predictor(Model_full_foldername)

  prediction=predictor.predict_proba(df[text_column].tolist())

  df_prediction = pd.DataFrame(prediction, columns = predictor.get_classes()) 
  df_prediction=df_prediction[[classname]]
  df_prediction.columns=['%s_%s'%(prediction_column,ind)]

  df['%s_%s'%(prediction_column,ind)]=df_prediction['%s_%s'%(prediction_column,ind)].to_numpy()
  df.to_csv(test_Messages_full_filename, header=True, index=False)
  
  

ts_end = time.time()
print((ts_end - ts_start)/60)

fit_onecycle_0 model is already processed
fit_onecycle_1 model is already processed
fit_onecycle_2 model is already processed
fit_onecycle_3 model is already processed
fit_onecycle_4 model is already processed
Processing fit_onecycle_5 model


Downloading:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Processing fit_onecycle_6 model
Processing fit_onecycle_7 model
Processing fit_onecycle_8 model
Processing fit_onecycle_9 model
457.1522248109182


## Average by all models/folds

In [ ]:
df[prediction_column]=df[prediction_column_lists].mean(axis=1)

In [ ]:
df.dtypes

Message_Id                int64
Timestamp                object
Topic_1st_Message        object
Parent_Id                 int64
Author_Id                 int64
Topic_Id                  int64
Topic                    object
Chapter_Id                int64
Chapter                  object
original_message         object
message                  object
cnt_childs                int64
cnt_immediate_childs      int64
author                   object
prediction_0            float64
prediction_1            float64
prediction_2            float64
prediction_3            float64
prediction_4            float64
prediction_5            float64
prediction_6            float64
prediction_7            float64
prediction_8            float64
prediction_9            float64
prediction              float64
dtype: object

## Fraction of messages which looks like "fedulya" messages in each author in the dataset

In [6]:
#only authors which score greater then threshold_fedulya_probability(0.9)
author_totals=df.groupby(['Author_Id']).size().reset_index(name='total_counts').sort_values(['total_counts'],ascending=False)
author_predicted=df[df[prediction_column]>threshold_fedulya_probability].groupby(['Author_Id']).size().reset_index(name='predicted_counts').sort_values(['predicted_counts'],ascending=False)
author_predicted=pd.merge(author_totals,author_predicted, left_on='Author_Id', right_on='Author_Id', how='inner')

In [9]:
#to get one author name per Author_Id (then can vary in messages)
Authors=pd.read_csv(Authors_full_filename, error_bad_lines=False, index_col=False)
author_predicted=pd.merge(Authors[['Author_Id','author']],author_predicted, left_on='Author_Id', right_on='Author_Id', how='inner')

In [10]:
#fraction of messages which looks like "fedulya"
author_predicted['fraction_%s_messages'%classname]=author_predicted['predicted_counts']/author_predicted['total_counts']

In [11]:
author_predicted.sort_values(['fraction_%s_messages'%classname],ascending=False, inplace=True)

## fedulya clones

In [12]:
author_predicted[author_predicted['fraction_fedulya_messages']>threshold_frc_fedulya_messages  ]

,Author_Id,author,total_counts,predicted_counts,fraction_fedulya_messages
100,186159,ФедЮля **,105,95,0.904762
125,250487,ФедюлЯ +,4530,3795,0.837748
7,17154,julia4evER **K**,88,66,0.750000
140,265057,Julia4eveR _,527,395,0.749526
145,266936,ФедЮлЯ _,243,180,0.740741
161,282313,Хули-я??? _,111,81,0.729730
163,283493,Хули-я?! *,2900,2044,0.704828


Let's check the rest of the authors with smaller score

In [13]:
author_predicted.head(30)

,Author_Id,author,total_counts,predicted_counts,fraction_fedulya_messages
100,186159,ФедЮля **,105,95,0.904762
125,250487,ФедюлЯ +,4530,3795,0.837748
7,17154,julia4evER **K**,88,66,0.750000
140,265057,Julia4eveR _,527,395,0.749526
145,266936,ФедЮлЯ _,243,180,0.740741
161,282313,Хули-я??? _,111,81,0.729730
163,283493,Хули-я?! *,2900,2044,0.704828
148,269514,ФедЮЛЯ *,1864,1259,0.675429
155,275933,Хули-Я? +,1256,846,0.673567
200,348002,gutalax _,269,171,0.635688


Looks like everybody including gutalax _ are clones with a fraction of messages which looks like "fedulya" >=0.6. In fact, they all have nick names related to russian name "Юля" (Julia) in different trabscription.The other group of authors with the fraction ~0.3 have a similar name to ФедЮля ** and might be also clones of the same author. Users from Жгучий перчик + to халоПЕньо + are also clones.

In [26]:
threshold_frc_fedulya_messages=0.04

In [27]:
author_predicted[author_predicted['fraction_fedulya_messages']>threshold_frc_fedulya_messages  ]

,Author_Id,author,total_counts,predicted_counts,fraction_fedulya_messages
100,186159,ФедЮля **,105,95,0.904762
125,250487,ФедюлЯ +,4530,3795,0.837748
7,17154,julia4evER **K**,88,66,0.750000
140,265057,Julia4eveR _,527,395,0.749526
145,266936,ФедЮлЯ _,243,180,0.740741
161,282313,Хули-я??? _,111,81,0.729730
163,283493,Хули-я?! *,2900,2044,0.704828
148,269514,ФедЮЛЯ *,1864,1259,0.675429
155,275933,Хули-Я? +,1256,846,0.673567
200,348002,gutalax _,269,171,0.635688


In [ ]:
#df.to_csv(test_Messages_full_filename, header=True, index=False)

### All ФедЮля ** users

In [28]:
fedulya = author_predicted[author_predicted['fraction_fedulya_messages']>threshold_frc_fedulya_messages  ].copy(deep=True)

In [29]:
Authors[((Authors['Author_Id'].isin(fedulya['Author_Id'].tolist())))][['Author_Id','author', 'cnt_messages', 'cnt_topics_started','first_message_ts','last_message_ts']].sort_values('first_message_ts',ascending=True)

,Author_Id,author,cnt_messages,cnt_topics_started,first_message_ts,last_message_ts
13120,250487,ФедюлЯ +,5571,14,2009-11-01 18:41:00,2010-02-22 16:43:00
10417,186159,ФедЮля **,119,0,2009-12-28 03:36:00,2010-02-19 03:26:00
13726,265057,Julia4eveR _,605,0,2010-02-28 02:49:00,2010-03-12 02:26:00
13808,266936,ФедЮлЯ _,287,0,2010-03-16 18:53:00,2010-03-23 23:41:00
13926,269514,ФедЮЛЯ *,2205,5,2010-04-05 23:20:00,2010-06-09 21:45:00
14165,275933,Хули-Я? +,1499,5,2010-06-10 22:53:00,2010-08-17 19:19:00
606,17154,julia4evER **K**,116,1,2010-06-20 01:37:00,2010-08-18 00:12:00
14371,282313,Хули-я??? _,123,1,2010-08-24 00:00:00,2010-08-27 02:34:00
14419,283493,Хули-я?! *,3300,8,2010-09-02 00:56:00,2010-12-07 01:30:00
15733,348002,gutalax _,317,1,2011-12-19 02:45:00,2011-12-22 10:24:00


The date range when users posted their messages are pretty consistent till some point. Some of date ranges are intersected but it's clear when they were created one by one till a gap 2012 - 2015. If the next users Федюля Иммодиум для анусов ** (466036) - Федюля - ЛопедиуМ для анусов + (482362) are indeed the same person, the gap explains why the fraction of similar messages are low in these users. Something changed in the life of the author as well as her style.

The next group of authors (from жгучий перчик F, 496807 to the last one, халоПЕньо +,	541604) have even less fraction of similar messages. But it looks like they were created even later  after a short gap in 6 months. Maybe belong to the sae user or maybe not.

In [21]:
#authors segmented into clusters - not all discovered authors can be in "CoreAuthors" due to low number of medium sized messages
CoreAuthors = pd.read_csv(CoreAuthors_full_filename, error_bad_lines=False, index_col=False) 

In [30]:
CoreAuthors[CoreAuthors['Author_Id'].isin(fedulya['Author_Id'].tolist())][['Author_Id','author','first_message_ts', 'last_message_ts','cnt_messages','cnt_medium_messages','normalized_kmeans']].sort_values('first_message_ts',ascending=True)

,Author_Id,author,first_message_ts,last_message_ts,cnt_messages,cnt_medium_messages,normalized_kmeans
1988,250487,ФедюлЯ +,2009-11-01 18:41:00,2010-02-22 16:43:00,5571,4530.0,1
1710,186159,ФедЮля **,2009-12-28 03:36:00,2010-02-19 03:26:00,119,105.0,1
2060,265057,Julia4eveR _,2010-02-28 02:49:00,2010-03-12 02:26:00,605,527.0,1
2068,266936,ФедЮлЯ _,2010-03-16 18:53:00,2010-03-23 23:41:00,287,243.0,1
2075,269514,ФедЮЛЯ *,2010-04-05 23:20:00,2010-06-09 21:45:00,2205,1864.0,1
2103,275933,Хули-Я? +,2010-06-10 22:53:00,2010-08-17 19:19:00,1499,1256.0,1
2116,282313,Хули-я??? _,2010-08-24 00:00:00,2010-08-27 02:34:00,123,111.0,1
2123,283493,Хули-я?! *,2010-09-02 00:56:00,2010-12-07 01:30:00,3300,2900.0,1
2244,348002,gutalax _,2011-12-19 02:45:00,2011-12-22 10:24:00,317,269.0,1
2359,466036,Федюля Иммодиум для анусов **,2015-03-23 04:31:00,2015-10-06 14:09:00,3776,3196.0,1


Well, they all belong to the same cluster and might be, indeed, the same person.

In [32]:
print('Total number of fedulya`s posts is %s'%Authors[((Authors['Author_Id'].isin(fedulya['Author_Id'].tolist())))]['cnt_messages'].sum())

Total number of fedulya`s posts is 24990


In [33]:
print('Total number of topics fedulya started is %s'%Authors[((Authors['Author_Id'].isin(fedulya['Author_Id'].tolist())))]['cnt_topics_started'].sum())

Total number of topics fedulya started is 53


They all quite active users with a lot of posts and new topics started. Each has a lot of feedback (responses to their posts).

In [34]:
tmp_df=df[df['Author_Id'].isin(fedulya['Author_Id'].tolist())][['Topic_Id','Author_Id']].groupby(['Topic_Id']).nunique().reset_index().sort_values('Author_Id', ascending=False)
tmp_df.columns=['Topic_Id','cnt_fedulya_authors']
tmp_df=tmp_df[tmp_df['cnt_fedulya_authors']>1]
print('Number of topics where fedulya is talking to herself using different users is %s'%len(tmp_df))

Number of topics where fedulya is talking to herself using different users is 48


*Need full dataset to check if she replies to her own messages. df is only named users (no anonymous) and intermediate replies can be skipped, so the chain of a conversation may have gaps.*

In [35]:
Authors[((Authors['Author_Id'].isin(fedulya['Author_Id'].tolist())))][['Author_Id','author','first_message_ts','fraction_of_messages_with_emojis','fraction_of_messages_with_images','fraction_of_messages_with_links','avg_message_words','avg_original_paragraphs','avg_actual_paragraphs','avg_paragraph_words']].sort_values('first_message_ts',ascending=True)

,Author_Id,author,first_message_ts,fraction_of_messages_with_emojis,fraction_of_messages_with_images,fraction_of_messages_with_links,avg_message_words,avg_original_paragraphs,avg_actual_paragraphs,avg_paragraph_words
13120,250487,ФедюлЯ +,2009-11-01 18:41:00,1.022976,0.000000,0.019207,23.979357,0.346796,0.957458,3.402983
10417,186159,ФедЮля **,2009-12-28 03:36:00,1.310924,0.000000,0.000000,29.621849,0.277311,1.176471,2.946439
13726,265057,Julia4eveR _,2010-02-28 02:49:00,1.266116,0.000000,0.038017,26.948760,0.323967,1.107438,4.633349
13808,266936,ФедЮлЯ _,2010-03-16 18:53:00,1.383275,0.000000,0.024390,32.857143,0.341463,1.191638,5.514579
13926,269514,ФедЮЛЯ *,2010-04-05 23:20:00,1.605896,0.000000,0.023129,26.301587,0.247619,1.166440,4.102231
14165,275933,Хули-Я? +,2010-06-10 22:53:00,1.533689,0.000000,0.018679,25.901268,0.160107,0.932622,3.775667
606,17154,julia4evER **K**,2010-06-20 01:37:00,1.379310,0.000000,0.051724,21.474138,0.129310,0.793103,3.007759
14371,282313,Хули-я??? _,2010-08-24 00:00:00,1.756098,0.000000,0.008130,25.634146,0.097561,1.073171,3.905285
14419,283493,Хули-я?! *,2010-09-02 00:56:00,1.718182,0.000000,0.014848,27.153636,0.189697,1.111515,4.022283
15733,348002,gutalax _,2011-12-19 02:45:00,0.599369,0.000000,0.000000,21.722397,0.264984,0.665615,2.525537


*Looks like using emojies makes original, not clean text in the model is more successfull then cleaned, without html tags text. Number of emojis changed at "gutalax _" user. Well, what if starting from "gutalax _" is an other user with clones?* 

In [36]:
Authors[( (Authors['Author_Id'].isin(fedulya['Author_Id'].tolist())))][['Author_Id','author','first_message_ts','cnt_messages','fraction_of_neg_rersponses','fraction_of_pos_rersponses']].sort_values('first_message_ts',ascending=True)

,Author_Id,author,first_message_ts,cnt_messages,fraction_of_neg_rersponses,fraction_of_pos_rersponses
13120,250487,ФедюлЯ +,2009-11-01 18:41:00,5571,0.444241,0.244973
10417,186159,ФедЮля **,2009-12-28 03:36:00,119,0.416667,0.258333
13726,265057,Julia4eveR _,2010-02-28 02:49:00,605,0.426332,0.242947
13808,266936,ФедЮлЯ _,2010-03-16 18:53:00,287,0.487889,0.228374
13926,269514,ФедЮЛЯ *,2010-04-05 23:20:00,2205,0.453051,0.234098
14165,275933,Хули-Я? +,2010-06-10 22:53:00,1499,0.449106,0.235901
606,17154,julia4evER **K**,2010-06-20 01:37:00,116,0.400000,0.243478
14371,282313,Хули-я??? _,2010-08-24 00:00:00,123,0.462185,0.210084
14419,283493,Хули-я?! *,2010-09-02 00:56:00,3300,0.462094,0.217509
15733,348002,gutalax _,2011-12-19 02:45:00,317,0.361775,0.259386


*Responses to her messages are rather negative then positive. 

In [37]:
Authors[((Authors['Author_Id'].isin(fedulya['Author_Id'].tolist())))][['Author_Id','author','first_message_ts','cnt_messages','fraction_of_tneg_messages','fraction_of_tpos_messages']].sort_values('first_message_ts',ascending=True)

,Author_Id,author,first_message_ts,cnt_messages,fraction_of_tneg_messages,fraction_of_tpos_messages
13120,250487,ФедюлЯ +,2009-11-01 18:41:00,5571,0.557837,0.198675
10417,186159,ФедЮля **,2009-12-28 03:36:00,119,0.628571,0.133333
13726,265057,Julia4eveR _,2010-02-28 02:49:00,605,0.540797,0.163188
13808,266936,ФедЮлЯ _,2010-03-16 18:53:00,287,0.596708,0.189300
13926,269514,ФедЮЛЯ *,2010-04-05 23:20:00,2205,0.554185,0.158798
14165,275933,Хули-Я? +,2010-06-10 22:53:00,1499,0.628185,0.102707
606,17154,julia4evER **K**,2010-06-20 01:37:00,116,0.590909,0.090909
14371,282313,Хули-я??? _,2010-08-24 00:00:00,123,0.594595,0.099099
14419,283493,Хули-я?! *,2010-09-02 00:56:00,3300,0.636207,0.107241
15733,348002,gutalax _,2011-12-19 02:45:00,317,0.568773,0.219331


*But her own messages are very negative.*